In [3]:
import pandas as pd
import sys; sys.path
import descartes
import geopandas as gpd
import matplotlib.pyplot as plt 
import numpy as np
import datetime as dt
import sys
from tqdm import tqdm_notebook as tqdm
import geopandas as gpd
from shapely.geometry import *
from shapely.ops import *
from geopandas import GeoDataFrame
from geopandas.tools import sjoin, overlay
from datetime import datetime, date, time
import os
import sqlalchemy
import numpy as np
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from datetime import datetime
from sqlalchemy import Column, text, create_engine
from sqlalchemy.types import TIMESTAMP
from sqlalchemy.dialects.mysql import BIGINT
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import glob

грузим остановки из базы

In [2]:
engine = create_engine('postgresql://makolesnikova@infraeconomy.com:zahceelaS4aewie7ouyohjoFuphah9Gu@84.201.144.84:54327/cei_samara')
meta = MetaData()
sti_val = Table('routes_vehicletrackpoint', meta, autoload=True, autoload_with=engine)
conn = engine.connect()
Session = sessionmaker(bind = engine)
session = Session()
first_stop=pd.read_sql('select distinct t2.mr_id, tt.rl_id, tt."ssId",tt.rc_orderby,t6.ss_name,t6.ss_lat,t6.ss_long \
  FROM public."transNavi_ReisStations"  tt \
left join public."transNavi_Reis_segments" t1 on tt.rl_id=t1.rl_id \
left join public."transNavi_MarshVariants" t2 on t1.mv_id=t2.mv_id \
left join public."transNavi_Marshes" t3 on t2.mr_id=t3.mr_id \
left join "transNavi_StopStation" t6 on tt."ssId"=t6."ssId" \
where rc_orderby=1', conn)

In [ ]:
tracks=pd.read_sql('select * from public."routes_vehicletrackpoint" where  mr_id=1 and date(timenav) =\'2019-10-01\'' ,conn)

In [161]:
path =".\\for_time\\bud\\"

In [164]:
files = [f for f in os.listdir(path) if "_3.csv" in f]

In [165]:
files

['2020-10-02_s2_3.csv',
 '2020-10-02_s3_3.csv',
 '2020-10-02_s5_3.csv',
 '2020-10-02_s6_3.csv',
 '2020-10-02_s7_3.csv']

In [166]:
df_4=pd.DataFrame()

In [167]:
for file in files:
    m = pd.read_csv(path+file, sep = '\t', encoding='windows-1251')
    df_4=df_4.append(m,ignore_index=True)
    del m

In [168]:
len(df_4)

1401

Анализ количества рейсов по маршуртам

In [38]:
df_4['time']=pd.to_datetime(df_4['timenav'])
df_4['hour']=df_4['time'].dt.hour
df_4['min']=df_4['time'].dt.minute
df_4_small_pd=df_4[['mr_id','tr_id','mr_name','hour','min','long','lat']].drop_duplicates()

In [39]:
first_stop=first_stop[['mr_id','rc_orderby','ssId','ss_lat','ss_long']].drop_duplicates()

In [40]:
first_stop.head()

,mr_id,rc_orderby,ssId,ss_lat,ss_long
0,1,1,80,53.384613,50.169362
1,1,1,552,53.187009,50.118634
2,1,1,13,53.303375,50.282002
5,1,1,66,53.303772,50.282262
7,1,1,1818,53.275044,50.266083


In [41]:
first_stop['ss_lat']=first_stop['ss_lat'].astype(float)
first_stop['ss_long']=first_stop['ss_long'].astype(float)

In [42]:
df_4_small_pd_merge=df_4_small_pd.merge(first_stop, on ='mr_id', how ='outer')

In [43]:
df_4_small_pd_merge.head()

,mr_id,tr_id,mr_name,hour,min,long,lat,rc_orderby,ssId,ss_lat,ss_long
0,105,200000006.0,48д,6.0,4.0,50.089637,53.126245,1,249,53.115204,50.063887
1,105,200000006.0,48д,6.0,4.0,50.089637,53.126245,1,38,53.192697,50.122152
2,105,200000006.0,48д,6.0,5.0,50.089548,53.126178,1,249,53.115204,50.063887
3,105,200000006.0,48д,6.0,5.0,50.089548,53.126178,1,38,53.192697,50.122152
4,105,200000006.0,48д,6.0,6.0,50.089800,53.126047,1,249,53.115204,50.063887


In [44]:
df_4_small_pd_merge=df_4_small_pd_merge[~pd.isna(df_4_small_pd_merge['long'])==True]

In [45]:
df_4_small_pd_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9520113 entries, 0 to 9520112
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   mr_id       int64  
 1   tr_id       float64
 2   mr_name     object 
 3   hour        float64
 4   min         float64
 5   long        float64
 6   lat         float64
 7   rc_orderby  int64  
 8   ssId        int64  
 9   ss_lat      float64
 10  ss_long     float64
dtypes: float64(7), int64(3), object(1)
memory usage: 871.6+ MB


In [46]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 * 1000 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [47]:
from tqdm import tqdm
tqdm.pandas()
# df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)
df_4_small_pd_merge['len'] = df_4_small_pd_merge.progress_apply(lambda x: haversine(x.long, x.lat, x.ss_long, x.ss_lat), axis = 1)
df_4_small_pd_merge

C:\Users\makol\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|█████████████████████████████████████████████████████████████████████| 9520113/9520113 [11:32<00:00, 13740.39it/s]


,mr_id,tr_id,mr_name,hour,min,long,lat,rc_orderby,ssId,ss_lat,ss_long,len
0,105,200000006.0,48д,6.0,4.0,50.089637,53.126245,1,249,53.115204,50.063887,2111.855909
1,105,200000006.0,48д,6.0,4.0,50.089637,53.126245,1,38,53.192697,50.122152,7700.558926
2,105,200000006.0,48д,6.0,5.0,50.089548,53.126178,1,249,53.115204,50.063887,2102.695185
3,105,200000006.0,48д,6.0,5.0,50.089548,53.126178,1,38,53.192697,50.122152,7709.379436
4,105,200000006.0,48д,6.0,6.0,50.089800,53.126047,1,249,53.115204,50.063887,2108.049813
...,...,...,...,...,...,...,...,...,...,...,...,...
9520108,118,704952059.0,18,19.0,13.0,50.204375,53.231027,1,185,53.230264,50.210125,392.015344
9520109,118,704952059.0,18,19.0,13.0,50.203232,53.231412,1,863,53.249921,50.264871,4589.159405
9520110,118,704952059.0,18,19.0,13.0,50.203232,53.231412,1,466,53.195719,50.290667,7046.161599
9520111,118,704952059.0,18,19.0,13.0,50.203232,53.231412,1,184,53.229654,50.208955,428.157659


In [48]:
df_4_small_pd_merge2=df_4_small_pd_merge[df_4_small_pd_merge['len']<100]

In [49]:
len(df_4_small_pd_merge2)

272963

In [50]:
df_4_small_pd_merge3=df_4_small_pd_merge2[['mr_id','hour','min','ssId','tr_id']].drop_duplicates()

In [51]:
len(df_4_small_pd_merge3)

144811

In [52]:
df_4_small_pd_merge3['time']=df_4_small_pd_merge3['hour']*60+df_4_small_pd_merge3['min']

In [53]:
df_4_small_pd_merge3=df_4_small_pd_merge3.sort_values(['mr_id','ssId','time'])

In [54]:
df_4_small_pd_merge3['tr_id2']=df_4_small_pd_merge3.groupby(['mr_id','ssId'])['tr_id'].shift(-1).astype('Int64')

In [55]:
df_4_small_pd_merge3['time2']=df_4_small_pd_merge3.groupby(['mr_id','ssId'])['time'].shift(-1).astype('Int64')

In [56]:
df_4_small_pd_merge3['delta']=df_4_small_pd_merge3['time2']-df_4_small_pd_merge3['time']

In [57]:
df_4_small_pd_merge3.head()

,mr_id,hour,min,ssId,tr_id,time,tr_id2,time2,delta
1958450,1,5.0,10.0,13,500000037.0,310.0,500000037,311,1.0
1958460,1,5.0,11.0,13,500000037.0,311.0,500000037,312,1.0
1958465,1,5.0,12.0,13,500000037.0,312.0,500000037,313,1.0
1958470,1,5.0,13.0,13,500000037.0,313.0,500000037,314,1.0
1958475,1,5.0,14.0,13,500000037.0,314.0,500000037,315,1.0


In [58]:
df_4_small_pd_merge4=df_4_small_pd_merge3[~((df_4_small_pd_merge3['tr_id2']==df_4_small_pd_merge3['tr_id2'])&
                                         (df_4_small_pd_merge3['delta']<20))]

In [59]:
len(df_4_small_pd_merge4)

4148

In [60]:
df_4_small_pd_merge4.to_csv('reises.csv')

In [61]:
path =".\\samaratransautogaz\\"

In [62]:
a=pd.read_file('stop_times.txt')

NameError: name 'read_file' is not defined

In [ ]:
poezdki=df_4_small_pd_merge3[['mr_id','']]

In [ ]:
poezdki_count_0=pd.DataFrame(poezdki_0.groupby(['mr_id']).agg(['count']))

Анализ пробега по маршрутам

In [ ]:
df_4['time']=pd.to_datetime(df_4['timenav'])
df_4['hour']=df_4['time'].dt.hour
df_4['min']=df_4['time'].dt.minute

df_4_small_pd=df_4[['mr_id','tr_id','mr_name','hour','min','long','lat']].drop_duplicates()

In [ ]:
df_4_small_pd.head()

In [ ]:
df_4_small_pd['rd_long_to'] = df_4_small_pd.groupby(['mr_id','tr_id'])['long'].shift(-1)

In [ ]:
df_4_small_pd['rd_lat_to'] = df_4_small_pd.groupby(['mr_id','tr_id'])['lat'].shift(-1)

In [ ]:
df_4_small_pd=df_4_small_pd[~pd.isna(df_4_small_pd['rd_long_to'])==True]

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 * 1000 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
df_cartesian=df_4_small_pd

In [ ]:
from tqdm import tqdm
tqdm.pandas()
# df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)
df_cartesian['len'] = df_cartesian.progress_apply(lambda x: haversine(x.long, x.lat, x.rd_long_to, x.rd_lat_to), axis = 1)
df_cartesian

In [ ]:
probeg=pd.DataFrame(df_cartesian.groupby(['mr_id']).agg({'len': np.sum})).reset_index()

In [ ]:
probeg.to_csv('probeg_bud.csv')

Закончили анализ пробега

Начали анализ скоростей

In [74]:
df_4_pick=df_4[df_4['timenav'].str.contains( ' 07:')]\
 .append(df_4[df_4['timenav'].str.contains( ' 08:')])\
 .append(df_4[df_4['timenav'].str.contains( ' 09:')])

In [82]:
df_4_day=df_4[df_4['timenav'].str.contains( ' 23:')]\
 .append(df_4[df_4['timenav'].str.contains( ' 22:')])\
 .append(df_4[df_4['timenav'].str.contains( ' 00:')])\
.append(df_4[df_4['timenav'].str.contains( ' 24:')])

In [83]:
df_4_pick2=df_4_pick[df_4_pick['speed']>0]
df_4_day=df_4_day[df_4_day['speed']>0]

In [77]:
df_4_pick['mr_id'].unique()

array([105, 159, 194, 155, 166, 164, 156, 192, 150, 125, 202, 160, 185,
       161, 196, 191, 184,  29, 268, 198, 183, 182, 154, 236, 162, 148,
       234, 250, 290, 233, 230, 262, 231, 243, 228, 289, 232, 227, 247,
       239, 241, 238, 245, 248, 244, 229, 259, 152,  48,  47,   1,  96,
       102,  32,  28,  23,  26,  17,  49,  64, 114,  81,  70, 113,  43,
        34,  38,  18,  20,  15,  30, 187,  68,  35,  36,  25, 100,  24,
        61,  62,  45,  39,  27, 199,  99,  59,  98,  73, 112,  14,  31,
        46,  60,  44, 136, 130, 142, 135, 144, 126, 141, 176, 134,  10,
       145, 127, 137, 143, 138, 140, 139, 129, 132, 146, 133, 151, 294,
       306, 122, 116, 121,  12, 117, 119,  97,  58,  71, 103, 115, 111,
       118, 120, 305], dtype=int64)

In [84]:
speed_by_marsh_pick=pd.DataFrame(df_4_pick.groupby(['mr_id']).agg({'speed': np.nanmean})).reset_index()
speed_by_marsh_day=pd.DataFrame(df_4_day.groupby(['mr_id']).agg({'speed': np.nanmean})).reset_index()

In [79]:
speed_by_marsh=speed_by_marsh_pick.merge(speed_by_marsh_day,on='mr_id', how='outer')

In [80]:
speed_by_marsh=speed_by_marsh.rename(columns={"speed_x": "speed_pick", "speed_y": "speed_day"})

In [85]:
speed_by_marsh_day.to_csv('speed_nigth.csv')

In [ ]:
poezdki = pd.read_csv('2020-08-03_all_4_first_stops_detected.csv',sep = ',', encoding='windows-1251')

In [ ]:
poezdki=poezdki[poezdki['order_by']==1]
poezdki=poezdki[['mr_id','time','Dir']]
poezdki=poezdki.sort_values(by=['mr_id','Dir','time'])
poezdki_0=poezdki[poezdki['Dir']==1]
poezdki_1=poezdki[poezdki['Dir']==0]


In [ ]:
poezdki_0=poezdki_0[['mr_id','Dir']]

In [ ]:
poezdki_1=poezdki_1[['mr_id','Dir']]

In [ ]:
poezdki['time2'] = poezdki.groupby(['mr_id','Dir'])['time'].shift(-1).astype('Int64')

In [ ]:
poezdki.head()

In [ ]:
poezdki_count_0=pd.DataFrame(poezdki_0.groupby(['mr_id']).agg(['count']))
poezdki_count_1=pd.DataFrame(poezdki_1.groupby(['mr_id']).agg(['count']))

In [ ]:
poezdki_count=poezdki_count_0.merge(poezdki_count_1,on='mr_id', how='outer')

In [ ]:
poezdki_count.to_csv('count_reises.csv')

In [ ]:
path = '.\\algoritm\\data\\'
p=pd.read_csv(path+'2020-08-03_s5_4.csv', sep = '\t', encoding='windows-1251')

In [ ]:
p2=p[p['mr_id']==32]

In [ ]:
p2.to_csv('mr_id_32.csv')

Выделенки

In [87]:
road=gpd.read_file('roads_39n_simp.shp')

In [89]:
vydel=road[road['projects']==1]

In [92]:
routes=gpd.read_file('Samara_201021.shp', encoding = 'utf-8')

In [93]:
routes.head()

,mr_name,tt_name,mv_id,rl_id,Status,route_name,route_type,LENGTH_GEO,geometry
0,1,Автобус,683,68365,changed,1 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,32.994318,"LINESTRING (50.17210 53.39407, 50.17146 53.393..."
1,1,Автобус,683,68366,changed,1 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,33.655301,"LINESTRING (50.11861 53.18702, 50.11867 53.186..."
2,1,Трамвай,752,75265,changed,1 ГОРОДСКОЙ МАРШРУТ,ТРАМВАЙ,13.056409,"LINESTRING (50.08176 53.17876, 50.08529 53.178..."
3,1,Трамвай,752,75266,changed,1 ГОРОДСКОЙ МАРШРУТ,ТРАМВАЙ,13.052657,"LINESTRING (50.23863 53.20343, 50.23548 53.202..."
4,101А,Автобус,374,37465,saved,101А ДАЧНЫЙ МАРШРУТ,АВТОБУС ПРИГОРОД/МЕЖГОРОД,16.728925,"LINESTRING (50.21728 53.07770, 50.21719 53.077..."


In [ ]:
vydel.byffer

Анализ времени работы на маршрутах

In [169]:
df_3=df_4

In [170]:
df_3.head()

,tr_id,mr_id,mr_name,graph,tabelnum,condnum,date,startshift,endshift
0,200000005,105,48д,17,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00
1,200000006,105,48д,21,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00
2,200000007,192,89,29,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00
3,200000010,159,96,10,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00
4,200000013,194,217,14,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00


In [171]:
df_3['start']=pd.to_datetime(df_3['startshift'])

In [172]:
df_3['start']=df_3['start'].dt.hour*60+df_3['start'].dt.minute

In [173]:
df_3['end']=pd.to_datetime(df_3['endshift'])
df_3['hour_']=df_3['end'].dt.hour
df_3.loc[df_3.hour_ ==0, 'hour_'] = 24
df_3.loc[df_3.hour_ ==1, 'hour_'] = 25
df_3.loc[df_3.hour_ ==2, 'hour_'] = 26
df_3['end']=df_3['hour_']*60+df_3['end'].dt.minute

In [174]:
df_3

,tr_id,mr_id,mr_name,graph,tabelnum,condnum,date,startshift,endshift,start,end,hour_
0,200000005,105,48д,17,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00,300,1500,25
1,200000006,105,48д,21,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00,300,1500,25
2,200000007,192,89,29,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00,300,1500,25
3,200000010,159,96,10,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00,300,1500,25
4,200000013,194,217,14,0,0,2020-10-02,2020-10-02 05:00:00,2020-10-03 01:00:00,300,1500,25
...,...,...,...,...,...,...,...,...,...,...,...,...
1396,704953064,12,4,223,3473,0,2020-10-02,2020-10-02 14:09:00,2020-10-03 00:31:00,849,1471,24
1397,704953067,119,17,223,1569,0,2020-10-02,2020-10-02 14:35:00,2020-10-02 20:51:00,875,1251,20
1398,704953068,122,20,902,4157,0,2020-10-02,2020-10-02 06:53:00,2020-10-02 11:46:00,413,706,11
1399,704953071,121,15,910,546,0,2020-10-02,2020-10-02 05:37:00,2020-10-02 19:31:00,337,1171,19


In [175]:
df_3['delta']=df_3['end']-df_3['start']

In [176]:
time_mr_id=pd.DataFrame(df_3.groupby(['mr_id']).agg({'delta': np.sum})).reset_index()

In [177]:
time_mr_id['delta']=time_mr_id['delta']/60

In [160]:
time_mr_id_bud=time_mr_id

In [178]:
time_mr_id=time_mr_id_bud.merge(time_mr_id,on='mr_id',how='outer')

In [179]:
time_mr_id.to_csv('time.csv')